In [1]:
!pip install --upgrade "google-meridian[colab] @ git+https://github.com/google/meridian.git"

  Cloning https://github.com/google/meridian.git to /private/var/folders/q5/fqrty7td6kl2szjpqjp9v56r0000gn/T/pip-install-_3nxww20/google-meridian_461e60b35bd046858c895a16735c0261
  Running command git clone --filter=blob:none --quiet https://github.com/google/meridian.git /private/var/folders/q5/fqrty7td6kl2szjpqjp9v56r0000gn/T/pip-install-_3nxww20/google-meridian_461e60b35bd046858c895a16735c0261
  Resolved https://github.com/google/meridian.git to commit 559103f4768906464162e25ca82aebd1a54c5ef5
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached arviz-0.20.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached altair-4.2.2-py3-none-any.whl.metadata (13 kB)
  Using cached immutabledict-4.2.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies ... one
  Getting requirements

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

ModuleNotFoundError: No module named 'numpy'

In [ ]:
coord_to_columns = load.CoordToColumns(
    time='date',
    # geo=None,  # No geographic column found in dataset
    controls=[],  # Removed clicks_Search and spend_Search as controls since they are now media
    # population=None,  # No explicit population column found
    kpi='revenue',  # Assuming revenue is the primary KPI
    # revenue_per_kpi=None,  # No explicit revenue per conversion column
    media=[
        'impressions_Facebook',
        'impressions_TV',
        'impressions_Youtube',
        'impressions_OOH',
        'impressions_Radio',
        'impressions_ProgammaticRT',
        'impressions_PMAX',
        'clicks_Search',  # Added clicks from search as media
    ],
    media_spend=[
        'spend_Facebook',
        'spend_TV',
        'spend_Youtube',
        'spend_OOH',
        'spend_Radio',
        'spend_ProgammaticRT',
        'spend_PMAX',
        'spend_Search',  # Added search spend as media spend
    ],
    organic_media=None,  # No organic media impression columns found
    non_media_treatments=['promo1', 'promo2'],  # Considering promotions as treatments
)

In [ ]:
correct_media_to_channel = {
    'impressions_Facebook': 'Facebook',
    'impressions_TV': 'TV',
    'impressions_Youtube': 'Youtube',
    'impressions_OOH': 'OOH',
    'impressions_Radio': 'Radio',
    'impressions_ProgammaticRT': 'ProgammaticRT',
    'impressions_PMAX': 'PMAX',
    'clicks_Search': 'Search',  # Clicks are now a media channel
}
correct_media_spend_to_channel = {
    'spend_Facebook': 'Facebook',
    'spend_TV': 'TV',
    'spend_Youtube': 'Youtube',
    'spend_OOH': 'OOH',
    'spend_Radio': 'Radio',
    'spend_ProgammaticRT': 'ProgammaticRT',
    'spend_PMAX': 'PMAX',
    'spend_Search': 'Search',  # Search spend is now included as media spend
}


In [ ]:
loader = load.CsvDataLoader(
    csv_path="/content/weekly2_monday_aligned.csv",
    kpi_type='revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()

In [ ]:
roi_mu = 0.15     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

In [ ]:
%%time
mmm.sample_prior(5000)
mmm.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=1000)

In [ ]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

In [ ]:
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()

In [ ]:
mmm_summarizer = summarizer.Summarizer(mmm)

In [ ]:
filepath = '/Users/arturodz/Development/simmmulator/'
start_date = '2020-01-01'
end_date = '2023-12-20'
mmm_summarizer.output_model_results_summary('summary_output.html', filepath, start_date, end_date)